In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('taz_calc.csv')
df.head()

,OID_,objectid,taz1454,district,county,gacres,SHAPE_Leng,SHAPE_Area,OBJECTID_1,taz1454_1,Shape_Area_1,AREA,PERCENTAGE
0,0,1,566,11,Santa Clara,1708.683056,0.097550,-0.000444,NaN,NaN,NaN,2.692192e+06,38.933797
1,1,2,565,11,Santa Clara,486.200669,0.054171,-0.000126,NaN,NaN,NaN,1.187353e+06,60.345733
2,7,8,570,11,Santa Clara,229.441293,0.034794,-0.000060,NaN,NaN,NaN,1.302642e+05,14.029284
3,68,69,394,9,Santa Clara,530.497558,0.046852,-0.000138,NaN,NaN,NaN,5.773166e+05,26.891365
4,69,70,450,9,Santa Clara,529.105063,0.047786,-0.000137,NaN,NaN,NaN,9.857419e+05,46.036630


In [4]:
df.drop(['OID_','objectid','OBJECTID_1', 'taz1454_1', 'Shape_Area_1'], axis=1, inplace=True)
df.PERCENTAGE.fillna(0, inplace=True)
df.head()

,taz1454,district,county,gacres,SHAPE_Leng,SHAPE_Area,AREA,PERCENTAGE
0,566,11,Santa Clara,1708.683056,0.097550,-0.000444,2.692192e+06,38.933797
1,565,11,Santa Clara,486.200669,0.054171,-0.000126,1.187353e+06,60.345733
2,570,11,Santa Clara,229.441293,0.034794,-0.000060,1.302642e+05,14.029284
3,394,9,Santa Clara,530.497558,0.046852,-0.000138,5.773166e+05,26.891365
4,450,9,Santa Clara,529.105063,0.047786,-0.000137,9.857419e+05,46.036630


In [5]:
df.query('PERCENTAGE >= 50').groupby('county').count()['PERCENTAGE']

county
Alameda           39
Contra Costa       9
San Francisco    103
San Mateo         15
Santa Clara       37
Solano             1
Name: PERCENTAGE, dtype: int64

In [6]:
df.set_index('taz1454', inplace=True)
df.head()

,district,county,gacres,SHAPE_Leng,SHAPE_Area,AREA,PERCENTAGE
taz1454,,,,,,,
566,11,Santa Clara,1708.683056,0.097550,-0.000444,2.692192e+06,38.933797
565,11,Santa Clara,486.200669,0.054171,-0.000126,1.187353e+06,60.345733
570,11,Santa Clara,229.441293,0.034794,-0.000060,1.302642e+05,14.029284
394,9,Santa Clara,530.497558,0.046852,-0.000138,5.773166e+05,26.891365
450,9,Santa Clara,529.105063,0.047786,-0.000137,9.857419e+05,46.036630


In [7]:
len(df)

1454

In [8]:
df.PERCENTAGE[df.PERCENTAGE >=100] = 100

C:\Users\sreichert\AppData\Local\Temp\ipykernel_26700\1904252604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.PERCENTAGE[df.PERCENTAGE >=100] = 100


In [33]:
df['gacres'].isna().sum()

0

In [50]:
df.rename(columns={'PERCENTAGE': 'transit_access'}, inplace=True)

## Data Pull
Pulling Data from ABAG open data portal API. Data links and metadata is available below. Only selected 2005 forecasts of variables.

Land Use: <https://opendata.mtc.ca.gov/datasets/MTC::plan-bay-area-2040-forecast-land-use-and-transportation/about>

Employment: <https://opendata.mtc.ca.gov/datasets/MTC::plan-bay-area-2040-forecast-employment/about>

Households: <https://opendata.mtc.ca.gov/datasets/MTC::plan-bay-area-2040-forecast-households/about>

Demographics (age only, not pulled): <https://opendata.mtc.ca.gov/datasets/MTC::plan-bay-area-2040-forecast-population-and-demographics/about>

In [9]:
import json
import requests

In [10]:
# Function pulls data from ESRI api which returns a nested json file and sets index to remain consistent
def LoadData(url):
    r = requests.get(url)
    data=json.loads(r.text)
    new_df = pd.json_normalize(data['features'])
    new_df.columns = new_df.columns.str.replace("attributes.","")
    new_df.set_index('zoneid', inplace=True)
    return new_df

In [39]:
df_lu = LoadData('https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/pba2040_forecast_landuse_transportation/FeatureServer/0/query?where=1%3D1&outFields=zoneid,areatype05,ciacre05,resacre05,sfdu05,mfdu05,terminal05,oprkcst05,prkcst05&outSR=4326&f=json')
df_household = LoadData('https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/pba2040_forecast_households/FeatureServer/0/query?where=1%3D1&outFields=zoneid,tothh05,q1hhinc05,q4hhinc05,q2hhinc05,q3hhinc05&outSR=4326&f=json')
df_emp = LoadData('https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/pba2040_forecast_employment/FeatureServer/0/query?where=1%3D1&outFields=zoneid,totemp05,empres05,retempn05,fpsempn05,herempn05,mwtempn05,agrempn05,othempn05&outSR=4326&f=json')


C:\Users\sreichert\AppData\Local\Temp\ipykernel_26700\1910248710.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df.columns = new_df.columns.str.replace("attributes.","")
C:\Users\sreichert\AppData\Local\Temp\ipykernel_26700\1910248710.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df.columns = new_df.columns.str.replace("attributes.","")
C:\Users\sreichert\AppData\Local\Temp\ipykernel_26700\1910248710.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df.columns = new_df.columns.str.replace("attributes.","")


## Land Use Processing

In [12]:
df_lu.head()

,areatype05,ciacre05,resacre05,sfdu05,mfdu05,terminal05,oprkcst05,prkcst05
zoneid,,,,,,,,
1,0,15,0,1,42,5.63188,716.0,218.0
301,4,97,492,2053,244,1.12131,0.0,0.0
251,2,44,152,432,2366,2.21316,0.0,0.0
201,3,59,66,787,1451,2.36899,0.0,0.0
151,3,1,26,395,153,2.25776,0.0,0.0


In [18]:
type_dict = {0:'Core', 1:'CBD', 2:'UrbanBusiness', 3: 'Urban', 4: 'Suburban', 5: 'Rural'}

In [41]:
df_lu['type'] = df_lu['areatype05'].apply(lambda x: type_dict[x])
df_lu['residential'] = np.select([df_lu['resacre05'] > df_lu['ciacre05']], [1], default=0)
df_lu['multifamily'] = np.select([df_lu['mfdu05']>df_lu['sfdu05']], [1], default=0)
df_lu = df_lu.join(df['gacres'])
df_lu['res_density'] = (df_lu[['mfdu05','sfdu05']].sum(axis=1)/df_lu['gacres'])
df_lu.head()

,areatype05,ciacre05,resacre05,sfdu05,mfdu05,terminal05,oprkcst05,prkcst05,type,residential,multifamily,gacres,res_density
zoneid,,,,,,,,,,,,,
1,0,15,0,1,42,5.63188,716.0,218.0,Core,0,1,32.530647,1.321830
301,4,97,492,2053,244,1.12131,0.0,0.0,Suburban,1,0,1297.232415,1.770693
251,2,44,152,432,2366,2.21316,0.0,0.0,UrbanBusiness,1,1,441.833064,6.332709
201,3,59,66,787,1451,2.36899,0.0,0.0,Urban,1,1,431.035742,5.192145
151,3,1,26,395,153,2.25776,0.0,0.0,Urban,1,0,95.444815,5.741538


In [43]:
Land_Use = pd.get_dummies(df_lu, prefix='LU_type', columns=['type']).drop(['areatype05','ciacre05','resacre05','sfdu05', 'mfdu05','gacres'], axis=1)
Land_Use.head()

,terminal05,oprkcst05,prkcst05,residential,multifamily,res_density,LU_type_CBD,LU_type_Core,LU_type_Rural,LU_type_Suburban,LU_type_Urban,LU_type_UrbanBusiness
zoneid,,,,,,,,,,,,
1,5.63188,716.0,218.0,0,1,1.321830,0,1,0,0,0,0
301,1.12131,0.0,0.0,1,0,1.770693,0,0,0,1,0,0
251,2.21316,0.0,0.0,1,1,6.332709,0,0,0,0,0,1
201,2.36899,0.0,0.0,1,1,5.192145,0,0,0,0,1,0
151,2.25776,0.0,0.0,1,0,5.741538,0,0,0,0,1,0


## Employment

In [16]:
df_emp.head()

,totemp05,empres05,retempn05,fpsempn05,herempn05,mwtempn05,agrempn05,othempn05
zoneid,,,,,,,,
51,833,2163,26,204,467,5,0,130
52,986,1361,18,122,374,138,0,333
53,880,52,39,29,598,18,0,196
54,685,3340,127,130,292,25,0,110
55,225,1677,21,100,80,15,0,10


In [45]:
df_emp = df_emp.join(df['gacres'])
df_emp['emp_density'] = df_emp['totemp05'] / df_emp['gacres']
df_emp.head()

,totemp05,empres05,retempn05,fpsempn05,herempn05,mwtempn05,agrempn05,othempn05,gacres
zoneid,,,,,,,,,
51,833,2163,26,204,467,5,0,130,171.530894
52,986,1361,18,122,374,138,0,333,2465.800635
53,880,52,39,29,598,18,0,196,1763.913987
54,685,3340,127,130,292,25,0,110,432.829446
55,225,1677,21,100,80,15,0,10,155.881173


In [47]:
df_emp.head()

,totemp05,empres05,retempn05,fpsempn05,herempn05,mwtempn05,agrempn05,othempn05,gacres,emp_density
zoneid,,,,,,,,,,
51,833,2163,26,204,467,5,0,130,171.530894,4.856268
52,986,1361,18,122,374,138,0,333,2465.800635,0.399870
53,880,52,39,29,598,18,0,196,1763.913987,0.498891
54,685,3340,127,130,292,25,0,110,432.829446,1.582610
55,225,1677,21,100,80,15,0,10,155.881173,1.443407


## Household Income

In [17]:
df_household.head()

,tothh05,q1hhinc05,q4hhinc05,q2hhinc05,q3hhinc05
zoneid,,,,,
1,37,29,0,7,1
2,134,100,3,25,6
3,247,182,5,47,13
4,69,52,1,13,3
5,480,352,11,92,25


In [24]:
df_household.columns[1:]

Index(['q1hhinc05', 'q4hhinc05', 'q2hhinc05', 'q3hhinc05'], dtype='object')

In [54]:
df_household['hhinc_quartile'] = df_household[df_household.columns[1:]].idxmax(axis=1).str.replace("hhinc05","")
df_household.head()

,tothh05,q1hhinc05,q4hhinc05,q2hhinc05,q3hhinc05,hhinc_quartile
zoneid,,,,,,
1,37,29,0,7,1,q1
2,134,100,3,25,6,q1
3,247,182,5,47,13,q1
4,69,52,1,13,3,q1
5,480,352,11,92,25,q1


In [56]:
primary_hhinc = pd.get_dummies(df_household['hhinc_quartile'], prefix='hhinc_')

## Joining Data

Varibles to Select:
- Land Use: All (time to vehicle storage, short term parking cost, long term parking costs, residential or commercial, single or multifamily, dummies for taz type)
- Employment: emp_density (employees/acre)
- Households: dummies for dominant household income quartile

In [58]:
joined_df = df[['transit_access','district','county']].join(Land_Use)
joined_df = joined_df.join(df_emp['emp_density'])
joined_df = joined_df.join(primary_hhinc)
joined_df.head()

,transit_access,district,county,terminal05,oprkcst05,prkcst05,residential,multifamily,res_density,LU_type_CBD,LU_type_Core,LU_type_Rural,LU_type_Suburban,LU_type_Urban,LU_type_UrbanBusiness,emp_density,hhinc__q1,hhinc__q2,hhinc__q3,hhinc__q4
taz1454,,,,,,,,,,,,,,,,,,,,
566,38.933797,11,Santa Clara,1.20229,0.0,0.0,1,1,1.007794,0,0,0,1,0,0,0.938149,0,1,0,0
565,60.345733,11,Santa Clara,1.49836,0.0,0.0,1,1,3.749481,0,0,0,1,0,0,2.190454,0,0,1,0
570,14.029284,11,Santa Clara,1.70474,0.0,0.0,1,0,4.807330,0,0,0,0,1,0,1.778233,1,0,0,0
394,26.891365,9,Santa Clara,1.40512,0.0,0.0,1,0,2.695583,0,0,0,0,1,0,3.751195,1,0,0,0
450,46.036630,9,Santa Clara,1.45681,0.0,0.0,0,1,3.284792,0,0,0,0,1,0,5.445043,1,0,0,0


In [65]:
joined_df.to_csv('../TAZ_database.csv')

1454